# Evolving a Lunar Lander with differentiable Genetic Programming

## Installation
To install the required libraries run the command:

In [1]:
#!pip install -r requirements.txt

## Imports
Imports from the standard genepro-multi library are done here. Any adjustments (e.g. different operators) should be made in the notebook. For example:

```
class SmoothOperator(Node):
  def __init__(self):
    super(SmoothOperator,self).__init__()
    self.arity = 1
    self.symb = "SmoothOperator"

  def _get_args_repr(self, args):
    return self._get_typical_repr(args,'before')

  def get_output(self, X):
    c_outs = self._get_child_outputs(X)
    return np.smoothOperation(c_outs[0])

  def get_output_pt(self, X):
    c_outs = self._get_child_outputs_pt(X)
    return torch.smoothOperation(c_outs[0])
```

In [2]:
import gymnasium as gym

from genepro.node_impl import *
from genepro.evo import Evolution
from genepro.node_impl import Constant
from genepro.variation import subtree_crossover, coeff_mutation
from genepro.multitree import Multitree

import torch
import torch.optim as optim

import random
import os
import copy
from collections import namedtuple, deque

import matplotlib.pyplot as plt
from matplotlib import animation

import numpy
import math

## Reinforcement Learning Setup
Here we first setup the Gymnasium environment. Please see https://gymnasium.farama.org/environments/box2d/lunar_lander/ for more information on the environment. 

Then a memory buffer is made. This is a buffer in which state transitions are stored. When the buffer reaches its maximum capacity old transitions are replaced by new ones.

A frame buffer is initialised used to later store animation frames of the environment.

In [3]:
env = gym.make("LunarLander-v2", render_mode="rgb_array")

In [4]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

    def __iadd__(self, other):
      self.memory += other.memory
      return self 

    def __add__(self, other):
      self.memory = self.memory + other.memory 
      return self

In [5]:
frames = []

## Fitness Function

Here you get to be creative. The default setup evaluates 5 episodes of 300 frames. Think of what action to pick and what fitness function to use. The Multi-tree takes an input of $n \times d$ where $n$ is a batch of size 1.

In [6]:
def gaussian_coeff_mutation(multitree : Multitree, optimize : bool, prob_coeff_mut : float= 1, temp : float=0.25) -> Node:
  """
  Applies gaussian coefficient mutations to constant nodes 

  Parameters
  ----------
  tree : Node
    the tree to which coefficient mutations are applied
  prob_coeff_mut : float, optional
    the probability with which coefficients are mutated (default is 0.25)
  temp : float, optional
    "temperature" that indicates the strength of coefficient mutation, it is relative to the current value (i.e., v' = v + temp*abs(v)*N(0,1))

  Returns
  -------
  Node
    the tree after coefficient mutation (it is the same as the tree in input)
  """
  for tree in multitree.children:
      coeffs = [n for n in tree.get_subtree() if type(n) == Constant]
      for c in coeffs:
        # decide wheter it should be applied
        if random.uniform(0, 1) < prob_coeff_mut:
          if not hasattr(c, 'sigma'):
              sample = np.random.normal(0, 0.1**2)
              setattr(c, 'sigma', max(math.exp(sample), 10**(-16)))
          else:
              v = c.get_value()
              # update the value by +- temp relative to current value
              # sample = random.gauss(0, 0.1)
              new_v = np.random.normal(v, c.sigma**2)
        
              c.sigma = max(c.sigma * math.exp(np.random.normal(0, 0.1**2)), 10**(-16))
              # new_v = v + sample #temp*np.abs(v)*randn()
              c.set_value(new_v)
  
  # multitree.children[r] = tree
  return multitree

In [7]:
def DQN_coeff_opt(multitree : Multitree, optimize : bool, prob_coeff_mut : float= 1, temp : float=0.25, ) -> Node:

    if not optimize:
        return multitree
        
    batch_size = 128
    GAMMA = 0.99
    
    constants = multitree.get_subtrees_consts()
    
    if len(constants)>0:
      optimizer = optim.AdamW(constants, lr=1e-3, amsgrad=True)
    
    for _ in range(100):
    
      if len(constants)>0 and len(evo.memory)>batch_size:
        target_tree = copy.deepcopy(multitree)
    
        transitions = evo.memory.sample(batch_size)
        batch = Transition(*zip(*transitions))
        
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                            batch.next_state)), dtype=torch.bool)
    
        non_final_next_states = torch.cat([s for s in batch.next_state
                                                   if s is not None])
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
    
        state_action_values = multitree.get_output_pt(state_batch).gather(1, action_batch)
        next_state_values = torch.zeros(batch_size, dtype=torch.float)
        with torch.no_grad():
          next_state_values[non_final_mask] = target_tree.get_output_pt(non_final_next_states).max(1)[0].float()
    
        expected_state_action_values = (next_state_values * GAMMA) + reward_batch
        
        criterion = nn.SmoothL1Loss()
        loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
       
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(constants, 100)
        optimizer.step()

    return multitree

In [8]:
def fitness_function_pt(multitree, num_episodes=5, episode_duration=300, render=False, ignore_done=False):
  memory = ReplayMemory(10000)
  rewards = []

  for _ in range(num_episodes):
    # get initial state of the environment
    observation = env.reset()
    observation = observation[0]
    
    for _ in range(episode_duration):
      if render:
        frames.append(env.render())

      input_sample = torch.from_numpy(observation.reshape((1,-1))).float()
      
      action = torch.argmax(multitree.get_output_pt(input_sample))
      observation, reward, terminated, truncated, info = env.step(action.item())
      rewards.append(reward)
      output_sample = torch.from_numpy(observation.reshape((1,-1))).float()
      memory.push(input_sample, torch.tensor([[action.item()]]), output_sample, torch.tensor([reward]))
      if (terminated or truncated) and not ignore_done:
        break

  fitness = np.sum(rewards) / num_episodes
  
  return fitness, memory

## Evolution Setup
Here the leaf and internal nodes are defined. Think about the odds of sampling a constant in this default configurations. Also think about any operators that could be useful and add them here. 

Adjust the population size (multiple of 8 if you want to use the standard tournament selection), max generations and max tree size to taste. Be aware that each of these settings can increase the runtime.

In [9]:
num_features = env.observation_space.shape[0]
leaf_nodes = [Feature(i) for i in range(num_features)]
leaf_nodes = leaf_nodes + [Constant()] # Think about the probability of sampling a coefficient
internal_nodes = [Plus(),Minus(),Times(),Div(),Square(),Sqrt()] #Add your own operators here

num_experiments = 10

## Evolve
Running this cell will use all the settings above as parameters

In [ ]:
import pickle

for i in range(num_experiments):
    evo = Evolution(
      fitness_function_pt, internal_nodes, leaf_nodes,
      4,
      pop_size=48,
      max_gens=40,
      max_tree_size=31,
      coeff_opts = [{"fun":gaussian_coeff_mutation, "rate": 1}],
      n_jobs=8,
      verbose=True
    )
    
    evo.evolve()

    file_path = f'best_gp{i+1}.pickle'
    # Open the file in binary mode
    with open(file_path, 'wb') as file:
        # Serialize and write the variable to the file
        pickle.dump(evo.best_of_gens[-1], file)

gen: 1,	best of gen fitness: -72.301,	best of gen size: 17
gen: 2,	best of gen fitness: -34.155,	best of gen size: 16
gen: 3,	best of gen fitness: -67.388,	best of gen size: 16
gen: 4,	best of gen fitness: -50.036,	best of gen size: 16
gen: 5,	best of gen fitness: -27.086,	best of gen size: 28
gen: 6,	best of gen fitness: -20.093,	best of gen size: 28
gen: 7,	best of gen fitness: -8.528,	best of gen size: 28
gen: 8,	best of gen fitness: 26.106,	best of gen size: 28
gen: 9,	best of gen fitness: -6.829,	best of gen size: 28
gen: 10,	best of gen fitness: -19.438,	best of gen size: 28
gen: 11,	best of gen fitness: 30.091,	best of gen size: 30
gen: 12,	best of gen fitness: -32.311,	best of gen size: 29
gen: 13,	best of gen fitness: 39.075,	best of gen size: 28
gen: 14,	best of gen fitness: -2.899,	best of gen size: 28
gen: 15,	best of gen fitness: -29.717,	best of gen size: 28
gen: 16,	best of gen fitness: -4.430,	best of gen size: 28
gen: 17,	best of gen fitness: 18.356,	best of gen size: 

In [ ]:
# Save model
# import pickle

# file_path = 'best_gp.pickle'

# # Open the file in binary mode
# with open(file_path, 'wb') as file:
#     # Serialize and write the variable to the file
#     pickle.dump(evo.best_of_gens[-1], file)

# Test

In [ ]:
def get_test_score(tree):
    rewards = []

    for i in range(10):
      # get initial state
      observation = env.reset(seed=i)
      observation = observation[0]

      for _ in range(500):    
        # build up the input sample for GP
        input_sample = torch.from_numpy(observation.reshape((1,-1))).float()
        # get output (squeezing because it is encapsulated in an array)
        output = tree.get_output_pt(input_sample)
        action = torch.argmax(tree.get_output_pt(input_sample))
        observation, reward, terminated, truncated, info = env.step(action.item())
        rewards.append(reward)


        output_sample = torch.from_numpy(observation.reshape((1,-1))).float()
        if (terminated or truncated):
            break

    fitness = np.sum(rewards)
    
    return fitness / 10

for i in range(num_experiments):
    print(f"For model {i+1}: \n")
    with open(f'best_gp{i+1}.pickle', 'rb') as input_file:
        best = pickle.load(input_file)
    print(best.get_readable_repr())
    print("test score:")
    print(get_test_score(best))

## Make an animation
Here the best evolved individual is selected and one episode is rendered. Make sure to save your lunar landers over time to track progress and make comparisons.

In [ ]:
frames = []

# gist to save gif from https://gist.github.com/botforge/64cbb71780e6208172bbf03cd9293553
def save_frames_as_gif(frames, path='./', filename='evolved_lander.gif'):
  plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)
  patch = plt.imshow(frames[0])
  plt.axis('off')
  def animate(i):
      patch.set_data(frames[i])
  anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
  anim.save(path + filename, writer='imagemagick', fps=60)

frames = []
fitness_function_pt(best, num_episodes=1, episode_duration=500, render=True, ignore_done=False)
env.close()
save_frames_as_gif(frames)

## Play animation

<img src="evolved_lander.gif" width="750">

## Optimisation
The coefficients in the multi-tree aren't optimised. Here Q-learning (taken from https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) is used to optimise the weights further. Incorporate coefficient optimisation in training your agent(s). Coefficient Optimisation can be expensive. Think about how often you want to optimise, when, which individuals etc.

### Question 3

Bring more elaborated ideas (also taking inspiration from the scientific literature) about how the provided GP library could be improved using coefficient optimisation. Optimisation can be expensive, so consider what individuals or what portion of the population should be optimised. Important: At this stage, each idea should be formalised into a motivating hypothesis (e.g., “Assuming that . . . happens, then we expect an improvement by doing . . . ”.


In [12]:
batch_size = 128
GAMMA = 0.99

for i in range(num_experiments):
    with open(f'best_gp{i+1}.pickle', 'rb') as input_file:
        best = pickle.load(input_file)
    constants = best.get_subtrees_consts()
    
    if len(constants)>0:
      optimizer = optim.AdamW(constants, lr=1e-3, amsgrad=True)
    
    for _ in range(500):
    
      if len(constants)>0 and len(evo.memory)>batch_size:
        target_tree = copy.deepcopy(best)
    
        transitions = evo.memory.sample(batch_size)
        batch = Transition(*zip(*transitions))
        
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                            batch.next_state)), dtype=torch.bool)
    
        non_final_next_states = torch.cat([s for s in batch.next_state
                                                   if s is not None])
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
    
        state_action_values = best.get_output_pt(state_batch).gather(1, action_batch)
        next_state_values = torch.zeros(batch_size, dtype=torch.float)
        with torch.no_grad():
          next_state_values[non_final_mask] = target_tree.get_output_pt(non_final_next_states).max(1)[0].float()
    
        expected_state_action_values = (next_state_values * GAMMA) + reward_batch
        
        criterion = nn.SmoothL1Loss()
        loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
       
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(constants, 100)
        optimizer.step()
    print(f"For model {i+1}: \n")
    print(best.get_readable_repr())
    print(get_test_score(best))
    

For model 1: 

['x_1', '(sqrt(abs(sqrt(abs(x_0))))*(sqrt(abs(sqrt(abs(x_2))))+(((((x_1*(x_4*x_4))-(x_0-x_3))/sqrt(abs((sqrt(abs(x_5))*x_2))))*x_6)+x_2)))', 'sqrt(abs((sqrt(abs(((((x_0+x_0)-x_5)+(x_5+sqrt(abs(x_0))))*x_3)))*x_3)))', '(((x_3+(x_6)**2)+(sqrt(abs(x_0))/(((x_6)**2)**2+x_2)))/(((x_4/x_5)*(x_2/x_6))-(x_6)**2))']
-72.32330500435073
For model 2: 

['(((sqrt(abs(x_1)))**2-((x_4-x_3))**2)-sqrt(abs((x_6/((x_7)**2-(((x_3-(sqrt(abs(x_1)))**2)-x_1))**2)))))', '((x_2+(x_3-x_2))*(sqrt(abs(x_1)))**2)', '(x_6/sqrt(abs(((((x_7+x_5)+((x_0/x_1)+sqrt(abs((sqrt(abs(((x_3+x_4))**2))/x_3)))))-sqrt(abs(sqrt(abs(sqrt(abs(sqrt(abs(x_6))))))))))**2)))', '(((11.709433555603027)**2+(((((sqrt(abs(sqrt(abs(-1.961763620376587))))-(x_0-x_7))*(sqrt(abs((x_6/x_3)))+(x_3/x_4)))-(x_6*(5.717799186706543)**2))*x_7)+x_7))/x_5)']
39.280330654148216
For model 3: 

['(x_2*(((x_4-x_1)+x_7)-(((x_4-x_1)+(x_2-(sqrt(abs((x_7*(x_3+x_6))))-x_7)))-sqrt(abs(((sqrt(abs(sqrt(abs(x_7)))))**2)**2)))))', '(((x_2+x_6)/sqrt(abs((

In [ ]:
batch_size = 128
GAMMA = 0.99

for i in range(num_experiments):
    with open(f'best_gp{i+1}.pickle', 'rb') as input_file:
        best = pickle.load(input_file)
    constants = best.get_subtrees_consts()
    
    if len(constants)>0:
      optimizer = optim.AdamW(constants, lr=1e-3, amsgrad=True)
    
    for _ in range(500):
    
      if len(constants)>0 and len(evo.memory)>batch_size:
        target_tree = copy.deepcopy(best)
    
        transitions = evo.memory.sample(batch_size)
        batch = Transition(*zip(*transitions))
        
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                            batch.next_state)), dtype=torch.bool)
    
        non_final_next_states = torch.cat([s for s in batch.next_state
                                                   if s is not None])
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
    
        state_action_values = best.get_output_pt(state_batch).gather(1, action_batch)
        next_state_values = torch.zeros(batch_size, dtype=torch.float)
        with torch.no_grad():
          next_state_values[non_final_mask] = target_tree.get_output_pt(non_final_next_states).max(1)[0].float()
    
        expected_state_action_values = (next_state_values * GAMMA) + reward_batch
        
        criterion = nn.SmoothL1Loss()
        loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
       
        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(constants, 100)
        optimizer.step()
    print(f"For model {i+1}: \n")
    print(best.get_readable_repr())
    print(get_test_score(best))
    

In [ ]:
frames = []
fitness_function_pt(best, num_episodes=1, episode_duration=500, render=True, ignore_done=False)
env.close()
save_frames_as_gif(frames, filename='evolved_lander_RL.gif')

<img src="evolved_lander_RL.gif" width="750">